In [1]:
import torch
from torch.utils.data import DataLoader
import numpy as np

import importlib
import sys
sys.path.append('..')
import util
importlib.reload(util)
importlib.reload(util.general)
importlib.reload(util.data)
importlib.reload(util.nn)
importlib.reload(util.nn.unet3d)
import util

log_board = util.diagnostics.LogBoard('log_dir', 6005)
log_board.launch()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
chunk_size = 80
dataset = util.data.SenNet(chunk_size)
train_data = DataLoader(dataset, batch_size=1, shuffle=True)
frac_pos = dataset.labels[0].float().mean()

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.15.1 at http://localhost:6005/ (Press CTRL+C to quit)


In [ ]:
# SAVE CURRENT LOG DIR BEFORE RUNNING THIS
# log_board.clear('train')

In [4]:
epochs = 100
batch_size = 4
logger = log_board.get_logger('train')

model = util.UNet3D(
    Conv3d=util.nn.Conv3DNormed
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scaler = torch.cuda.amp.GradScaler()
# loss_fn = util.BinaryFocalLoss(alpha=0.9, gamma=2.0).to(device)
loss_fn = util.DiceLoss().to(device)

agg_dice = []
agg_loss = []
for epoch in range(epochs):
    for i, (x, y) in enumerate(train_data):
        step = epoch * len(train_data) + i
        x, y = x.float().to(device), y.float().to(device)
        x = (x - x.mean()) / x.std()
        # with torch.cuda.amp.autocast():
        pred_y = model(x)
        loss = loss_fn(pred_y[-1], y)
        loss /= batch_size
        agg_loss.append(loss.item())
        agg_dice.append(util.dice_coefficient(
            (pred_y[-1] > 0.5).float(), y
        ).item())
        scaler.scale(loss).backward()
        logger.add_scalars('stats',{
            'out_std': pred_y[-1].std(),
            'out_mean': pred_y[-1].mean(),
            'vessel_density': y.mean(),
            'pred_density': (pred_y[-1] > 0.5).float().mean(),
            # 'bce_min': bce.min(),
            # 'bce_max': bce.max(),
        }, step)

        if step != 0 and step % batch_size == 0: # backward after we've collected a full batch
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

            logger.add_scalar('loss', np.sum(agg_loss), step)
            logger.add_scalar('dice', np.mean(agg_dice), step)

            agg_dice = []
            agg_loss = []

In [10]:
del model, x, y
torch.cuda.empty_cache()

NameError: name 'model' is not defined

In [6]:
dataset.scans[0].shape

torch.Size([1, 2375, 1399, 1008])

In [5]:
x, y = next(iter(train_data))
x = x.half().to(device)
y = y.half().to(device)
with torch.cuda.amp.autocast():
    # with torch.no_grad():
    y_pred = model(x)
    loss = y_pred[-1].mean()
    print(loss.item())
    loss.backward()

436 78 615
569 679 322
28.328125


In [6]:
y_pred[-1].dtype

torch.float16

In [10]:
# x, y = next(iter(train_data))
x, y = x.half(), y.half()
print(y[0].max())
util.Display(
    x[0].cpu().squeeze(),
    # y[0].cpu().squeeze()
)(True)

tensor(0., dtype=torch.float16)


interactive(children=(IntSlider(value=0, description='idx_td', max=95), Output()), _dom_classes=('widget-inter…

interactive(children=(IntSlider(value=0, description='idx_side', max=95), Output()), _dom_classes=('widget-int…

In [16]:
y.count_nonzero()
y.numel()

1048576

In [8]:
model = util.UNet3D().to(device)